In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
def getOpticalFlow(video):
    """Calculate dense optical flow of input video using GPU.
    Args:
        video: the input video with shape of [frames, height, width, channel]. dtype=np.array
    Returns:
        flows_x: the optical flow at x-axis, with the shape of [frames, height, width, channel]
        flows_y: the optical flow at y-axis, with the shape of [frames, height, width, channel]
    """
    # Initialize the list of optical flows
    gray_video = []
    for i in range(len(video)):
        img = cv2.cvtColor(video[i], cv2.COLOR_RGB2GRAY)
        gray_video.append(np.reshape(img, (224, 224, 1)))

    flows = []
    for i in range(0, len(video) - 1):
        # Calculate optical flow between each pair of frames
        prev_frame = torch.tensor(gray_video[i], device=device, dtype=torch.float32)
        next_frame = torch.tensor(gray_video[i + 1], device=device, dtype=torch.float32)
        flow = cv2.calcOpticalFlowFarneback(
            prev_frame.cpu().numpy(), next_frame.cpu().numpy(), None, 0.5, 3, 15, 3, 5, 1.2, cv2.OPTFLOW_FARNEBACK_GAUSSIAN
        )
        # Subtract the mean to eliminate the movement of the camera
        flow[..., 0] -= np.mean(flow[..., 0])
        flow[..., 1] -= np.mean(flow[..., 1])
        # Normalize each component in optical flow
        flow[..., 0] = cv2.normalize(flow[..., 0], None, 0, 255, cv2.NORM_MINMAX)
        flow[..., 1] = cv2.normalize(flow[..., 1], None, 0, 255, cv2.NORM_MINMAX)
        # Add into list
        flows.append(flow)

    # Padding the last frame as an empty array
    flows.append(np.zeros((224, 224, 2)))

    return np.array(flows, dtype=np.float32)


In [ ]:
def Save2Npy(file_dir, save_dir):
    """Transfer all the videos and save them into a specified directory
    Args:
        file_dir: source folder of target videos
        save_dir: destination folder of output .npy files
    """
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    # List the files
    videos = os.listdir(file_dir)
    for v in tqdm(videos):
        # Split video name
        video_name = v.split('.')[0]
        # Get src
        video_path = os.path.join(file_dir, v)
        # Get dest
        save_path = os.path.join(save_dir, video_name + '.npy')
        # Load and preprocess video
        try:
            data = Video2Npy(file_path=video_path, resize=(224, 224))
            if data is not None:
                data = np.uint8(data)
                # Save as .npy file
                np.save(save_path, data)
            else:
                print(f"Skipping {video_path} due to errors.")
        except Exception as e:
            print(f"Error processing {video_path}: {e}")

    return None


In [ ]:
source_path = "/home/islabworker3/tienvo/dataset_video"
target_path = "/home/islabworker3/tienvo/dangdat/data-video/source"

for f1 in ['train', 'val', 'test']:
    for f2 in ['normal', 'restricted']:
        path1 = os.path.join(source_path, f1, f2)
        path2 = os.path.join(target_path, f1, f2)
        Save2Npy(file_dir=path1, save_dir=path2)
